# <center> Degrees of Separation with Breadth-first Search </center>

In this project, **Breath-first Search** (BFS) algorithm was implemented in **Spark RDD** to find the degrees of separations between two given [Marvel Superheros](https://www.marvel.com/comics/characters).

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
# Create a new SparkConf object to configure the Spark application
conf = SparkConf().setMaster('local').setAppName('DegreesOfSeparation')
# Create a new SparkContext object with the specified configuration (conf)
sc = SparkContext(conf = conf)

### About the Datasets

The **Marvel-names.txt** dataset consists of two columns: **heroID** and **superhero names**. The heroID uniquely identifies each individual superhero in the dataset. The superhero names represent the names of the corresponding superheroes. The dataset provides a mapping between the heroIDs and their corresponding superhero names, allowing easy identification and reference to specific superheroes within the Marvel universe.

In [3]:
!head -n 10 ./data/Marvel-names.txt

1 "24-HOUR MAN/EMMANUEL"
2 "3-D MAN/CHARLES CHAN"
3 "4-D MAN/MERCURIO"
4 "8-BALL/"
5 "A"
6 "A'YIN"
7 "ABBOTT, JACK"
8 "ABCISSA"
9 "ABEL"
10 "ABOMINATION/EMIL BLO"


The **Marvel-graph.txt** dataset represents the network of Marvel superheroes based on their appearances together in the same comics. It is structured as a graph with each row containing a superhero's ID (heroID) followed by the IDs of other superheroes with whom they have appeared together in comics. The dataset helps establish connections between superheroes who have shared comic book appearances, reflecting their interactions and collaborations within the Marvel universe.

In [4]:
!head -n 5 ./data/Marvel-graph.txt

5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 
5989 4080 4264 4446 3779 2430 2297 6169 3530 3272 4282 6432 2548 4140 185 105 3878 2429 1334 4595 2767 3956 3877 4776 4946 3407 128 269 5775 5121 481 5516 4758 4053 1044 1602 3889 1535 6038 533 3986 
5982 217 595 1194 3308 2940 1815 794 1503 5197 859 5096 6039 2664 651 2244 528 284 1449 1097 1172 1092 108 3405 5204 387 4607 4545 3705 4930 1805 4712 4404 247 4754 4427 1845 536 5795 5978 533 3984 6056 
5983 1165 3836 4361 1282 716 4289 4646 6300 5084 2397 4454 1913 5861 5485 
5980 2731 3712 1587 6084 2472 2546 6313 875 859 323 2664 1469 522 2506 2919 2423 3624 5736 5046 1787 5776 3245 3840 2399 


### Define the functions for Breadth-first Search

In [5]:
# Function to convert each input record into the format (heroID, (connections, distance, color))
def convertToBFS(x):
    # Extract the hero ID from the input record and convert it to an integer
    heroID = int(x.split()[0])
    # Extract the list of connections from the input record and convert them to integers
    connections = [int(connection) for connection in x.split()[1:]]
    # Remove duplicate connections
    connections = list(set(connections))
    # Initialize the distance to a large value (effectively infinity)
    distance = 9999
    # Initialize the color of the hero to 'WHITE', indicating it has not been visited yet
    color = 'WHITE'
    
    # Check if the heroID is the source hero
    if heroID == sourceHeroID:
        # If the hero is the source hero, set the distance to 0 as it is the starting point
        distance = 0
        # Also set the color to 'GRAY', indicating that it has been discovered but not fully explored
        color = 'GRAY'
    
    return (heroID, (connections, distance, color))

In [6]:
# Function to create the initial RDD from the input file
def createStartingRdd(sc, file_path):
    # Read the input file containing the Marvel graph data and create an RDD
    RDD_lines = sc.textFile(file_path)
    # Map each line of the RDD to a new format (heroID, (connections, distance, color)) using the convertToBFS function
    return RDD_lines.map(convertToBFS)

In [7]:
# Function to perform the map operation for the RDD
def bfsMap(x):
    # Extract the hero ID, connections, distance, and color from the input record
    heroID = x[0]
    connections = x[1][0]
    distance = x[1][1]
    color = x[1][2]
    
    # Initialize an empty list to store the results of the BFS map phase
    result = []
    
    # Exploring the hero that's in 'GRAY' state in the current BFS iteration
    if color == 'GRAY':
        # Change the color of the current hero to 'BLACK', indicating it has been fully explored in this iteration
        color = 'BLACK'
        
        # Iterate through the connections of the current hero
        for connection in connections:
            # Create a new entry for each connection with an updated distance and the color set to 'GRAY'
            # This represents the exploration of the next level in the BFS
            newEntry = (connection, ([], distance + 1, 'GRAY'))
            # Append the new entry to the result list
            result.append(newEntry)
            # If the connection is the target hero, increment the hitCounter indicating that the target is reached
            if connection == targetHeroID:
                hitCounter.add(1)
    
    # Append the current hero to the result list as well
    result.append( (heroID, (connections, distance, color)) )
    
    # Return the result list containing the updated hero information and its connected nodes for the next BFS iteration
    return result

In [8]:
# Function to perform the reduce operation for the RDD
def bfsReduce(x, y):
    # Extract the connections, distance, and color from the input records x and y that share the same key (i.e. heroID)
    connections_1 = x[0]
    connections_2 = y[0]
    distance_1 = x[1]
    distance_2 = y[1]
    color_1 = x[2]
    color_2 = y[2]
    
    # Merge the connections from both records and remove duplicates
    connections = connections_1 + connections_2
    connections = list(set(connections))
    
    # Determine the minimum distance from the source hero to this hero
    if distance_1 <= distance_2:
        distance = distance_1
    else:
        distance = distance_2
    
    # Determine the final color for this hero
    if (color_1 == 'WHITE' and color_2 == 'GRAY') or (color_1 == 'GRAY' and color_2 == 'WHITE'):
        color = 'GRAY'
    elif (color_1 == 'BLACK' and color_2 == 'GRAY') or (color_1 == 'GRAY' and color_2 == 'BLACK'):
        color = 'BLACK'
    else:
        color = color_1
        
    # Return a new tuple as value representing the combined information for this hero
    return (connections, distance, color)

Read **Marvel-names.txt** data into a RDD for superhero queries.

In [9]:
# Read the input file 'Marvel-names.txt' containing hero names and IDs and create an RDD
RDD_heros = sc.textFile('./data/Marvel-names.txt')
# Map each line of the RDD to a new format (heroID, heroName)
RDD_heros = RDD_heros.map(lambda x: (int(x.split()[0]), ' '.join(x.split()[1:])[1:-1]) )

# Preview the first 10 records
RDD_heros.collect()[:10]

[(1, '24-HOUR MAN/EMMANUEL'),
 (2, '3-D MAN/CHARLES CHAN'),
 (3, '4-D MAN/MERCURIO'),
 (4, '8-BALL/'),
 (5, 'A'),
 (6, "A'YIN"),
 (7, 'ABBOTT, JACK'),
 (8, 'ABCISSA'),
 (9, 'ABEL'),
 (10, 'ABOMINATION/EMIL BLO')]

In [10]:
# Function to perform a search for a superhero's ID based on their name or vice versa
def searchHero(heroName = None, heroID = None):
    # Search for a superhero's ID based on name
    if heroName:
        return RDD_heros.filter(lambda x: heroName.upper() in x[1]).collect()
    
    # Search for a superhero's name based on ID
    if heroID:
        return RDD_heros.filter(lambda x: x[0] == heroID).collect()[0][1]

In [11]:
# Main logic function for executing the BFS algorithm to find degrees of separations between two given superheros
def bfsMain(sc, sourceHeroID, targetHeroID):
    # Print the names of the source hero and target hero based on their IDs
    sourceHeroName = searchHero(heroID = sourceHeroID)
    targetHeroName = searchHero(heroID = targetHeroID)
    print('Source hero: ' + sourceHeroName)
    print('Target hero: {}\n'.format(targetHeroName))
    
    # Initialize the hitCounter to 0 as no target hero has been hit yet
    global hitCounter
    hitCounter = sc.accumulator(0)
    # Initialize the iteration variable to 1, which represents the index for current iteration
    iteration = 1
    
    # Create the initial RDD to start the BFS algorithm
    network_file_path = './data/Marvel-graph.txt'
    iterationRDD = createStartingRdd(sc, network_file_path)
    
    # Continue BFS iterations until the target hero is found or all connected heros have been explored
    while True:
        print('Running BFS iteration {}'.format(iteration))
        
        # Perform the map operation to explore the connections of each 'GRAY' hero
        mapRDD = iterationRDD.flatMap(bfsMap)
        
        # Count the number of records in the mapRDD, 
        # forcing the RDD to be evaluated and updating the hitCounter
        print('Processing {} values.'.format(mapRDD.count()))
        
        # Check if the target hero has been hit (i.e., reached during exploration)
        if hitCounter.value > 0:
            print('Target character successfully hit! From {} different direction(s).'.format(hitCounter.value))
            print('\n{} is {} degree(s) of separation away from {}'.format(targetHeroName, iteration, sourceHeroName))
            break
        else:
            # If the target hero has not been hit, prepare for the next BFS iteration
            # Increment the index for iteration
            iteration += 1
            # Combine the records with same heroID into a single record in the updated iterationRDD
            iterationRDD = mapRDD.reduceByKey(bfsReduce)

### Test to find the degrees of separation between two given superheros

Find the degrees of separation between two random superheros

In [12]:
# Define the ID of the source hero and the target hero
sourceHeroID = 5301
targetHeroID = 41

# Call the bfsMain function to perform BFS
bfsMain(sc, sourceHeroID, targetHeroID)

Source hero: SPERZEL, ANTON
Target hero: AGAMEMNON

Running BFS iteration 1
Processing 6594 values.
Running BFS iteration 2
Processing 10683 values.
Running BFS iteration 3
Processing 248780 values.
Running BFS iteration 4
Processing 96187 values.
Target character successfully hit! From 2 different direction(s).

AGAMEMNON is 4 degree(s) of separation away from SPERZEL, ANTON


Find the degrees of separation between ***Deadpool*** and ***Quicksilver***

In [13]:
# Search for Deadpool's ID
searchHero(heroName = 'deadpool')

[(1397, 'DEADPOOL/JACK/WADE W')]

In [14]:
# Search for Quicksilver's ID
searchHero(heroName = 'quicksilver')

[(4454, 'QUICKSILVER/PIETRO M'),
 (4455, 'QUICKSILVER DOPPELGA'),
 (4456, 'QUICKSILVER | MUTANT')]

In [15]:
# Define the ID of the source hero and the target hero
sourceHeroID = 1397
targetHeroID = 4454

# Call the bfsMain function to perform BFS
bfsMain(sc, sourceHeroID, targetHeroID)

Source hero: DEADPOOL/JACK/WADE W
Target hero: QUICKSILVER/PIETRO M

Running BFS iteration 1
Processing 6797 values.
Running BFS iteration 2
Processing 49756 values.
Target character successfully hit! From 74 different direction(s).

QUICKSILVER/PIETRO M is 2 degree(s) of separation away from DEADPOOL/JACK/WADE W
